In [332]:
#import des librairies

import os
import re
import PyPDF2
import csv
import numpy as np
import pandas as pd

#extraction  pdf --> csv

def extract_times_from_pdf(file_path):
    pdf_reader = PyPDF2.PdfReader(file_path)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

    times = re.findall(r'\d{2}:\d{2}:\d{2}', text)
    return times

def extract_pos_nom_from_pdf(file_path):
    pdf_reader = PyPDF2.PdfReader(file_path)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

    lines = text.split('\n')
    pos = []
    nom = []
    for line in lines:
        if line.strip():
            match = re.match(r'^(\d+)\s+(\d+)\s+(.*?)\s+\d{2}:\d{2}:\d{2}$', line)
            if match:
                pos.append(match.group(1))
                nom.append(match.group(3))
    return pos, nom


directory = '.'

t_officiel_5k = []
t_officiel_10k = []
t_officiel_15k = []
t_officiel_20k = []
t_officiel_25k = []
t_officiel_30k = []
t_officiel_35k = []
t_officiel_40k = []
t_officiel_full = []

for filename in os.listdir(directory):
    if filename.startswith("Results") and filename.endswith(".pdf"):
        match = re.search(r'Results(\d+)k?\.pdf', filename)
        if match:
            distance = int(match.group(1))
            pdf_file_path = os.path.join(directory, filename)
            times = extract_times_from_pdf(pdf_file_path)

            if distance == 5:
                t_officiel_5k.extend(times)
            elif distance == 10:
                t_officiel_10k.extend(times)
            elif distance == 15:
                t_officiel_15k.extend(times)
            elif distance == 20:
                t_officiel_20k.extend(times)
            elif distance == 25:
                t_officiel_25k.extend(times)
            elif distance == 30:
                t_officiel_30k.extend(times)
            elif distance == 35:
                t_officiel_35k.extend(times)
            elif distance == 40:
                t_officiel_40k.extend(times)
            elif distance == 42:  
                t_officiel_full.extend(times)

pos, nom = extract_pos_nom_from_pdf(os.path.join(directory, 'Results.pdf'))

csv_file = 'results1.csv'

fieldnames = ['pos', 'nom']
for distance in [5, 10, 15, 20, 25, 30, 35, 40, 0]:
    fieldnames.append(f't_officiel_{distance}k')

data = [{'pos': pos, 'nom': nom} for pos, nom in zip(pos, nom)]

times_lists = [
    t_officiel_5k,
    t_officiel_10k,
    t_officiel_15k,
    t_officiel_20k,
    t_officiel_25k,
    t_officiel_30k,
    t_officiel_35k,
    t_officiel_40k,
    t_officiel_full,
]

for i, row in enumerate(data):
    for j, distance in enumerate(fieldnames[2:]):
        if j < len(times_lists) and i < len(times_lists[j]):
            row[distance] = times_lists[j][i]

with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

print("CSV file created successfully.")

CSV file created successfully.


In [333]:
#Lire le fichier CSV
df = pd.read_csv('results1.csv')

#Renommer la colonne
df = df.rename(columns={'t_officiel_0k': 'temps_tot'})

#Écrire le DataFrame mis à jour dans un nouveau fichier CSV
df.to_csv('results1.csv', index=False)

In [334]:
#vitesse pour tous les coureurs
distance_km = 42.195  
df['Vitesse (km/h)'] = distance_km / (pd.to_timedelta(df['temps_tot']) / pd.Timedelta('1 hour'))


df['Vitesse (km/h)'] = df['Vitesse (km/h)'].round(1)


In [335]:
#negative split
negativ_split = pd.to_timedelta(df['t_officiel_40k']) - pd.to_timedelta(df['t_officiel_20k'])

df['negativ_split'] = negativ_split.dt.total_seconds().div(60).mul(100).astype(int).astype(str).str.zfill(4)
df['negativ_split'] = df['negativ_split'].str[:2] + ':' + df['negativ_split'].str[2:]

df.head(10)


,pos,nom,t_officiel_5k,t_officiel_10k,t_officiel_15k,t_officiel_20k,t_officiel_25k,t_officiel_30k,t_officiel_35k,t_officiel_40k,temps_tot,Vitesse (km/h),negativ_split
0,1,TADESSE ABRAHAM,00:14:45,00:29:30,00:44:34,00:59:29,01:14:08,01:28:55,01:43:44,01:58:35,02:05:01,20.3,59:10
1,2,COLLINS KIPKURUI KIPKORIR,00:14:46,00:29:31,00:44:35,00:59:30,01:14:08,01:28:55,01:43:44,01:59:15,02:06:44,20.0,59:75
2,3,EDMOND KIPNGETICH,00:14:46,00:29:31,00:44:35,00:59:30,01:14:08,01:28:55,01:44:00,02:00:08,02:07:21,19.9,60:63
3,4,ILIASS AOUANI,00:14:46,00:29:31,00:44:35,00:59:30,01:14:27,01:29:54,01:45:19,02:00:56,02:08:05,19.8,61:43
4,5,BERHANE TESFAY,00:14:46,00:29:31,00:44:35,00:59:30,01:14:34,01:29:54,01:45:57,02:02:40,02:10:19,19.4,63:16
5,6,GEOFFREY KIMUTAI KOECH,00:14:46,00:29:31,00:44:36,00:59:30,01:14:34,01:30:18,01:46:39,02:03:36,02:11:03,19.3,64:09
6,7,SAHLESILASSIE NIGUSSIE BEKELE,00:14:47,00:29:47,00:44:53,00:59:43,01:15:47,01:32:37,01:49:36,02:06:02,02:13:10,19.0,66:31
7,8,GADISA BEKELE GUTAMA,00:14:58,00:29:52,00:45:22,01:01:08,01:16:59,01:33:20,01:50:26,02:08:28,02:16:25,18.6,67:33
8,9,ARTUR BOSSY ANGUERA,00:15:01,00:29:53,00:45:31,01:01:40,01:17:34,01:33:23,01:50:38,02:10:19,02:19:11,18.2,68:65
9,10,OLLE MEIJER,00:16:35,00:33:01,00:49:37,01:06:10,01:22:25,01:38:34,01:54:56,02:11:56,02:19:28,18.2,65:76


In [350]:
#vitesse moyenne pour les premiers 10
print("premieres 10 places 🏆 :")
print(df[['nom', 'temps_tot']].head(10))

premieres 10 places 🏆 :
                             nom temps_tot
0                TADESSE ABRAHAM  02:05:01
1      COLLINS KIPKURUI KIPKORIR  02:06:44
2              EDMOND KIPNGETICH  02:07:21
3                  ILIASS AOUANI  02:08:05
4                 BERHANE TESFAY  02:10:19
5         GEOFFREY KIMUTAI KOECH  02:11:03
6  SAHLESILASSIE NIGUSSIE BEKELE  02:13:10
7           GADISA BEKELE GUTAMA  02:16:25
8            ARTUR BOSSY ANGUERA  02:19:11
9                    OLLE MEIJER  02:19:28


In [337]:
#vitesse moyenne 
print("Vitesse moyenne des coureurs 🏃 :")
print(df[['nom', 'Vitesse (km/h)']])

Vitesse moyenne des coureurs 🏃 :
                               nom  Vitesse (km/h)
0                  TADESSE ABRAHAM            20.3
1        COLLINS KIPKURUI KIPKORIR            20.0
2                EDMOND KIPNGETICH            19.9
3                    ILIASS AOUANI            19.8
4                   BERHANE TESFAY            19.4
5           GEOFFREY KIMUTAI KOECH            19.3
6    SAHLESILASSIE NIGUSSIE BEKELE            19.0
7             GADISA BEKELE GUTAMA            18.6
8              ARTUR BOSSY ANGUERA            18.2
9                      OLLE MEIJER            18.2
10                DANIEL ?OCHOWSKI            18.2
11                 DEGITU AZIMERAW            18.1
12                      RALF ULMER            18.1
13               SHUKO GENEMO WOTE            17.9
14  LUIS FIGUEROA GARCIA-CUBILLANA            17.8
15         RUBEN PALOMEQUE BARRERA            17.8
16            MESERET DINKE MELEKA            17.7
17                   ADRIEN BETRON            17.

In [351]:
#moyenne de la vitesse pour touts les runners
moyenne_vitesse = sum(df['Vitesse (km/h)']) / len(df['Vitesse (km/h)'])

# Affichage du résultat
print(f"La vitesse moyenne de tout les coureurs est d'environ {moyenne_vitesse} km/h.")

La vitesse moyenne de tout les coureurs est d'environ 17.741666666666664 km/h.


In [339]:
#Negative split
print("Negative split 📈")
print(df[['nom', 'negativ_split']])

Negative split 📈
                               nom negativ_split
0                  TADESSE ABRAHAM         59:10
1        COLLINS KIPKURUI KIPKORIR         59:75
2                EDMOND KIPNGETICH         60:63
3                    ILIASS AOUANI         61:43
4                   BERHANE TESFAY         63:16
5           GEOFFREY KIMUTAI KOECH         64:09
6    SAHLESILASSIE NIGUSSIE BEKELE         66:31
7             GADISA BEKELE GUTAMA         67:33
8              ARTUR BOSSY ANGUERA         68:65
9                      OLLE MEIJER         65:76
10                DANIEL ?OCHOWSKI         65:98
11                 DEGITU AZIMERAW         66:05
12                      RALF ULMER         66:43
13               SHUKO GENEMO WOTE         67:70
14  LUIS FIGUEROA GARCIA-CUBILLANA         68:33
15         RUBEN PALOMEQUE BARRERA         68:38
16            MESERET DINKE MELEKA         68:51
17                   ADRIEN BETRON         69:11
18            DAVID DALMAU NAVARRO         68:66
19 

In [340]:
#densité des coureurs par kilomètre
distance_km = 42.195 
num_coureurs = 20000  
densite_coureurs = int(num_coureurs / distance_km)

print(f"Densité des coureurs : {densite_coureurs:.2f} coureurs/km")


Densité de coureurs : 473.00 coureurs/km
